In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import pandas as pd
import datetime
from os.path import isfile, join
import time_hist2
from geospacepy import omnireader


### This script reads in FAST data and collates the solar wind and geomagnetic activity data using the ['geospacepy'](https://github.com/lkilcommons/geospacepy-lite) functionality


##### Define a function to recalculate the Newell Coupling Function

In [2]:
def NewellCF_calc(v,bz,by):
    # v expected in km/s
    # b's expected in nT    
    NCF = np.zeros_like(v)
    NCF.fill(np.nan)
    bt = np.sqrt(by**2 + bz**2)
    bztemp = bz
    bztemp[bz == 0] = .001
    #Caculate clock angle (theta_c = t_c)
    tc = np.arctan2(by,bztemp)
    neg_tc = bt*np.cos(tc)*bz < 0 
    tc[neg_tc] = tc[neg_tc] + np.pi
    sintc = np.abs(np.sin(tc/2.))
    NCF = (v**1.33333)*(sintc**2.66667)*(bt**0.66667)
    return NCF

##### Define a function to identify a nearest index based on provided value

In [3]:
# Function to find nearest indice in array (works with datetimes) - written for use with omnireader
def nearest(items, pivot):
    #return min(items, key=lambda x: abs(x - pivot))
    return np.argmin(abs(items - pivot))




##### Read in FAST data

In [4]:
file = 'FAST_NHSep1998_33Strangewayorbits__ion_electron.csv'
data = pd.read_csv(file)



In [5]:
data.head()

,date,j,je,n,jerr,jeerr,nerr,mlt,mlon,mlat,...,sza,mono,broad,diffuse,n_el,j_el,je_el,nerr_el,jerr_el,jeerr_el
0,1998-09-23 12:33:07.500,0.0,0.0,0.0,0.0,0.0,0.0,13.42003,83.05871,60.05458,...,53.74877,0.0,0.0,1.0,0.00702,7086131.5,0.04322,0.01584,27642998.0,0.31013
1,1998-09-23 12:33:10.000,0.0,0.0,0.0,0.0,0.0,0.0,13.42352,83.10271,60.11309,...,53.82582,0.0,0.0,1.0,0.00710,7385806.0,0.04656,0.02678,51650252.0,1.32208
2,1998-09-23 12:33:12.500,0.0,0.0,0.0,0.0,0.0,0.0,13.42704,83.14320,60.17195,...,53.90285,0.0,0.0,1.0,0.00680,7221613.0,0.04322,0.01107,14345626.0,0.04677
3,1998-09-23 12:33:15.000,0.0,0.0,0.0,0.0,0.0,0.0,13.43055,83.18753,60.23040,...,53.97986,0.0,0.0,1.0,0.00743,7374004.0,0.04472,0.15941,312797664.0,1.97284
4,1998-09-23 12:33:17.500,0.0,0.0,0.0,0.0,0.0,0.0,13.43055,83.18753,60.23040,...,53.97986,0.0,0.0,1.0,0.00743,7374004.0,0.04472,0.15941,312797664.0,1.97284


In [6]:
data.describe()

,j,je,n,jerr,jeerr,nerr,mlt,mlon,mlat,alt,sza,mono,broad,diffuse,n_el,j_el,je_el,nerr_el,jerr_el,jeerr_el
count,1.545100e+04,15451.000000,15451.000000,1.545100e+04,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,1.545100e+04,15451.000000,15451.000000,1.545100e+04,15451.00000
mean,7.116573e+07,-0.002325,26.824983,4.412504e+05,0.000907,0.155015,13.493442,-14.950340,73.271708,4051.538704,71.958155,0.039609,0.114362,0.911462,0.222100,1.995073e+08,0.592108,0.008433,1.105654e+07,0.11623
std,3.856346e+08,0.016543,120.782063,1.067733e+07,0.012606,0.876898,1.680633,114.897752,7.260317,71.653020,11.929052,0.209755,0.455880,0.284085,1.377586,1.753877e+09,13.709758,0.230617,4.081406e+08,4.44957
min,0.000000e+00,-0.321460,0.000000,0.000000e+00,0.000000,0.000000,6.289160,-179.994340,60.000600,3800.115720,44.466830,0.000000,0.000000,0.000000,0.000030,1.673316e+05,0.000100,0.000040,1.090949e+05,0.00014
25%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,12.303525,-124.147295,67.216490,4017.889280,63.050490,0.000000,0.000000,1.000000,0.005890,5.082262e+06,0.005820,0.001005,9.470063e+05,0.00468
50%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,13.326150,-33.196790,73.388310,4079.547850,71.890330,0.000000,0.000000,1.000000,0.013000,1.055654e+07,0.023950,0.002170,2.026530e+06,0.00898
75%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,14.372395,97.525170,78.496330,4107.068360,81.141610,0.000000,0.000000,1.000000,0.068310,5.362790e+07,0.062150,0.005190,4.540230e+06,0.02067
max,7.052062e+09,0.000000,2755.122560,1.011069e+09,0.933600,83.025510,17.976520,179.977450,89.454870,4116.432130,98.305310,2.000000,2.000000,1.000000,35.289470,5.018452e+10,410.085820,27.816470,4.927633e+10,497.65756


##### Account for temporal cyclical variations by converting time variables to cosine and sine terms

In [7]:
%%time

# Calculate doy and UT from datetime
ut = []
for d in range(len(data)):
    dt_now = (pd.to_datetime( data['date'][d] ))
    ut.append( (dt_now - dt_now.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds() )
    
    
day_of_year = [pd.to_datetime(data['date'].iloc[d]).timetuple().tm_yday for d in range(len(data))]



data['sin_ut'] = [np.sin(2*np.pi*(x/60.)/1440.) for x in ut]
data['cos_ut'] = [np.cos(2*np.pi*(x/60.)/1440.) for x in ut]

data['sin_doy'] = [np.sin(2*np.pi*x/365.) for x in day_of_year]
data['cos_doy'] = [np.cos(2*np.pi*x/365.) for x in day_of_year]

data['sin_mlt'] = [np.sin(2*np.pi*x/24.) for x in data.mlt]
data['cos_mlt'] = [np.cos(2*np.pi*x/24.) for x in data.mlt]







CPU times: user 4.6 s, sys: 18.7 ms, total: 4.62 s
Wall time: 4.63 s


##### Set the index to the datetime

In [8]:
data.index = pd.DatetimeIndex( data['date'] )



In [10]:
data.head()

,date,j,je,n,jerr,jeerr,nerr,mlt,mlon,mlat,...,je_el,nerr_el,jerr_el,jeerr_el,sin_ut,cos_ut,sin_doy,cos_doy,sin_mlt,cos_mlt
date,,,,,,,,,,,,,,,,,,,,,
1998-09-23 12:33:07.500,1998-09-23 12:33:07.500,0.0,0.0,0.0,0.0,0.0,0.0,13.42003,83.05871,60.05458,...,0.04322,0.01584,27642998.0,0.31013,-0.948362,0.317191,-0.991114,-0.133015,0.228966,0.973434
1998-09-23 12:33:10.000,1998-09-23 12:33:10.000,0.0,0.0,0.0,0.0,0.0,0.0,13.42352,83.10271,60.11309,...,0.04656,0.02678,51650252.0,1.32208,-0.933837,0.357698,-0.991114,-0.133015,0.229024,0.973421
1998-09-23 12:33:12.500,1998-09-23 12:33:12.500,0.0,0.0,0.0,0.0,0.0,0.0,13.42704,83.14320,60.17195,...,0.04322,0.01107,14345626.0,0.04677,-0.917584,0.397543,-0.991114,-0.133015,0.229083,0.973407
1998-09-23 12:33:15.000,1998-09-23 12:33:15.000,0.0,0.0,0.0,0.0,0.0,0.0,13.43055,83.18753,60.23040,...,0.04472,0.15941,312797664.0,1.97284,-0.899631,0.436651,-0.991114,-0.133015,0.229142,0.973393
1998-09-23 12:33:17.500,1998-09-23 12:33:17.500,0.0,0.0,0.0,0.0,0.0,0.0,13.43055,83.18753,60.23040,...,0.04472,0.15941,312797664.0,1.97284,-0.880012,0.474951,-0.991114,-0.133015,0.229142,0.973393


In [13]:
data.columns.to_list()


['date',
 'j',
 'je',
 'n',
 'jerr',
 'jeerr',
 'nerr',
 'mlt',
 'mlon',
 'mlat',
 'alt',
 'sza',
 'mono',
 'broad',
 'diffuse',
 'n_el',
 'j_el',
 'je_el',
 'nerr_el',
 'jerr_el',
 'jeerr_el',
 'sin_ut',
 'cos_ut',
 'sin_doy',
 'cos_doy',
 'sin_mlt',
 'cos_mlt']

##### Drop unwanted columns


In [9]:
data.drop(['date'], axis=1, inplace=True)



In [10]:
print('date range:\n  {} \n        to\n  {}'.format(data.index.min(),data.index.max()))

date range:
  1998-09-23 12:33:07.500000 
        to
  1998-09-26 11:40:17.500000


##### Create the 'OMNI interval' which will download locally and pull in the OMNI data over period specified

In [11]:
t_start = data.index.min() - datetime.timedelta(1)
t_end = data.index.max() + datetime.timedelta(1)


omniInt = omnireader.omni_interval(t_start,t_end,'5min', cdf_or_txt = 'txt')
omniInt_1hr = omnireader.omni_interval(t_start,t_end,'hourly', cdf_or_txt = 'txt')
    
    

Created interval between 1998-09-22 and 1998-09-27, cadence 5min, start index 76183, end index 77613
Created interval between 1998-09-22 and 1998-09-27, cadence hourly, start index 6349, end index 6468


In [12]:
epochs = omniInt['Epoch'] #time array for omni 5min data



In [14]:
# just a check that the 'nearest' function is working properly

print(pd.to_datetime(data.index[0]))

idx_chc = nearest(epochs,data.index[0])

print(idx_chc)
print(epochs[idx_chc])

1998-09-23 12:33:07.500000
288
1998-09-23 12:35:00


##### Collate the solar wind data with each FAST observation in 'data'

In [17]:
# %%time

t_start = data.index.min() - datetime.timedelta(1)
t_end = data.index.max() + datetime.timedelta(1)

omniInt = omnireader.omni_interval(t_start,t_end,'5min', cdf_or_txt = 'txt')
omniInt_1hr = omnireader.omni_interval(t_start,t_end,'hourly', cdf_or_txt = 'txt')
    
# Get solar wind 5-minute data
epochs = omniInt['Epoch'] #time array for omni 5min data
Bx,By,Bz,AE,SymH = omniInt['BX_GSE'],omniInt['BY_GSM'],omniInt['BZ_GSM'],omniInt['AE_INDEX'], omniInt['SYM_H']
vsw,psw = omniInt['flow_speed'], omniInt['Pressure']
borovsky_reader = omnireader.borovsky(omniInt)
borovsky = borovsky_reader()

newell = NewellCF_calc(vsw, Bz, By)
    
epochs_1hr = omniInt_1hr['Epoch'] #datetime timestamps
F107,Kp = omniInt_1hr['F10_INDEX'],omniInt_1hr['KP']

# Get solar wind derived data
SW_df = pd.DataFrame(data = np.column_stack((Bz,By,Bx,AE,SymH,vsw,psw,borovsky,newell)),
                     index = epochs,
                     columns=['Bz','By','Bx','AE','SymH','vsw','psw','borovsky','newell'])

# Calculate the time histories of the OMNI data
SW_new = time_hist2.time_history(SW_df)

# Resample the OMNI data to the cadence of the FAST data (i.e., 2.5 seconds)
SW_df_25s = SW_new.copy().resample('2500L').pad()

# Merge the FAST and OMNI data
complete_dataframe = data.join(SW_df_25s,how='left')
complete_dataframe.to_csv('fast_sw_join_strangeway.csv')

#------------------------------------------------------------------------------------------------------------
# BELOW IS THE DEPRECATED APPROACH OF IDENTIFYING THE INDEX IN THE OMNI DATA FOR EACH POINT IN THE FAST DATA
#   --> COMPUTATIONALLY EXPENSIVE
#------------------------------------------------------------------------------------------------------------

# # inquire_idx = [nearest(epochs,d) for d in data.index[0:10]]
# inquire_idx = [nearest(epochs,pd.to_datetime(d)) for d in data.index]
# Bx = Bx[inquire_idx]
# By = By[inquire_idx]
# Bz = Bz[inquire_idx]
# AE = AE[inquire_idx]
# SymH = SymH[inquire_idx]
# vsw = vsw[inquire_idx]
# psw = psw[inquire_idx]
# borovsky = borovsky[inquire_idx]
# newell = newell[inquire_idx]

# # Get solar wind derived data
# SW_df = pd.DataFrame(data = np.column_stack((Bz,By,Bx,AE,SymH,vsw,psw,borovsky,newell)),
#                      index = data.index,
#                      columns=['Bz','By','Bx','AE','SymH','vsw','psw','borovsky','newell'])


# SW_new = time_hist2.time_history(SW_df)

# # Concatenate with FAST dataframe
# complete_dataframe = pd.merge(data, SW_new, how='outer', left_index=True, right_on=index, validate='many_to_one')




Created interval between 1998-09-22 and 1998-09-27, cadence 5min, start index 76183, end index 77613
Created interval between 1998-09-22 and 1998-09-27, cadence hourly, start index 6349, end index 6468
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP


In [25]:
complete_dataframe.columns.to_list()

['j',
 'je',
 'n',
 'jerr',
 'jeerr',
 'nerr',
 'mlt',
 'mlon',
 'mlat',
 'alt',
 'sza',
 'mono',
 'broad',
 'diffuse',
 'n_el',
 'j_el',
 'je_el',
 'nerr_el',
 'jerr_el',
 'jeerr_el',
 'sin_ut',
 'cos_ut',
 'sin_doy',
 'cos_doy',
 'sin_mlt',
 'cos_mlt',
 'Bz',
 'By',
 'Bx',
 'AE',
 'SymH',
 'vsw',
 'psw',
 'borovsky',
 'newell',
 'Bz_6hr',
 'By_6hr',
 'Bx_6hr',
 'AE_6hr',
 'SymH_6hr',
 'vsw_6hr',
 'psw_6hr',
 'borovsky_6hr',
 'newell_6hr',
 'Bz_5hr',
 'By_5hr',
 'Bx_5hr',
 'AE_5hr',
 'SymH_5hr',
 'vsw_5hr',
 'psw_5hr',
 'borovsky_5hr',
 'newell_5hr',
 'Bz_3hr',
 'By_3hr',
 'Bx_3hr',
 'AE_3hr',
 'SymH_3hr',
 'vsw_3hr',
 'psw_3hr',
 'borovsky_3hr',
 'newell_3hr',
 'Bz_1hr',
 'By_1hr',
 'Bx_1hr',
 'AE_1hr',
 'SymH_1hr',
 'vsw_1hr',
 'psw_1hr',
 'borovsky_1hr',
 'newell_1hr',
 'Bz_45min',
 'By_45min',
 'Bx_45min',
 'AE_45min',
 'SymH_45min',
 'vsw_45min',
 'psw_45min',
 'borovsky_45min',
 'newell_45min',
 'Bz_30min',
 'By_30min',
 'Bx_30min',
 'AE_30min',
 'SymH_30min',
 'vsw_30min',
 'ps

In [26]:
complete_dataframe.describe()

,j,je,n,jerr,jeerr,nerr,mlt,mlon,mlat,alt,...,newell_10min,Bz_5min,By_5min,Bx_5min,AE_5min,SymH_5min,vsw_5min,psw_5min,borovsky_5min,newell_5min
count,1.545100e+04,15451.000000,15451.000000,1.545100e+04,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,...,1.545100e+04,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,15451.000000,14951.000000,1.545100e+04,1.545100e+04
mean,7.116573e+07,-0.002325,26.824983,4.412504e+05,0.000907,0.155015,13.493442,-14.950340,73.271708,4051.538704,...,6.792137e+06,320.804755,328.260605,317.241483,524.196880,-53.973076,3774.438347,2.849511,3.791412e+14,6.792217e+06
std,3.856346e+08,0.016543,120.782063,1.067733e+07,0.012606,0.876898,1.680633,114.897752,7.260317,71.653020,...,3.707631e+07,1770.129189,1768.770902,1770.777451,484.332661,50.615958,17598.120354,2.339863,2.073315e+15,3.707630e+07
min,0.000000e+00,-0.321460,0.000000,0.000000e+00,0.000000,0.000000,6.289160,-179.994340,60.000600,3800.115720,...,1.387340e+01,-22.120000,-11.670000,-14.850000,40.000000,-213.000000,346.100000,0.760000,3.789666e+02,1.387340e+01
25%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,12.303525,-124.147295,67.216490,4017.889280,...,4.466964e+03,-5.160000,0.100000,-9.570000,165.000000,-56.000000,457.900000,1.700000,1.546315e+04,4.466964e+03
50%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,13.326150,-33.196790,73.388310,4079.547850,...,6.824209e+03,-2.150000,4.580000,-7.270000,374.000000,-40.000000,527.000000,2.190000,2.741916e+04,7.117706e+03
75%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,14.372395,97.525170,78.496330,4107.068360,...,1.424439e+04,1.850000,9.450000,-3.510000,655.000000,-22.000000,664.400000,2.680000,4.573989e+04,1.460485e+04
max,7.052062e+09,0.000000,2755.122560,1.011069e+09,0.933600,83.025510,17.976520,179.977450,89.454870,4116.432130,...,2.095289e+08,9999.990000,9999.990000,9999.990000,2801.000000,-2.000000,99999.900000,14.830000,1.171622e+16,2.095289e+08
